# Get NL-Augmenter and install dependencies

## Copy git repository of NL-Augmenter

In [1]:
!git clone https://www.github.com/GEM-benchmark/NL-Augmenter

Cloning into 'NL-Augmenter'...
remote: Enumerating objects: 11928, done.
remote: Counting objects: 100% (599/599), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 11928 (delta 334), reused 378 (delta 201), pack-reused 11329
Receiving objects: 100% (11928/11928), 91.25 MiB | 21.22 MiB/s, done.
Resolving deltas: 100% (7782/7782), done.
Checking out files: 100% (773/773), done.


## Go to newly copied directory

In [2]:
cd NL-Augmenter

/content/NL-Augmenter


## Install requirements

In [4]:
!pip install -r requirements.txt --quiet

     |████████████████████████████████| 12.1 MB 229 kB/s 
     |████████████████████████████████| 12.7 MB 22.6 MB/s 
     |████████████████████████████████| 101 kB 9.3 MB/s 
     |████████████████████████████████| 280 kB 54.1 MB/s 
     |████████████████████████████████| 234 kB 69.2 MB/s 
     |████████████████████████████████| 2.2 MB 45.1 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 1.2 MB 65.0 MB/s 
     |████████████████████████████████| 138 kB 73.7 MB/s 
     |████████████████████████████████| 190 kB 70.5 MB/s 
     |████████████████████████████████| 73 kB 67 kB/s 
     |████████████████████████████████| 105 kB 60.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 167 kB 65.0 MB/s 
     |████████████████████████████████| 9.1 MB 41.4 MB/s 
     |████████████████████████████████| 628 kB 50.2 MB/s 
     |███████████████

# Data loading

##  Get dataloader and use it to load CommonGen dataset

In [5]:
from datasets import load_dataset

data = load_dataset("gem", "common_gen")

Dataset gem downloaded and prepared to /root/.cache/huggingface/datasets/gem/common_gen/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4. Subsequent calls will reuse this data.


## Explore data structure

In [6]:
data

DatasetDict({
    train: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 67389
    })
    validation: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 993
    })
    test: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 1497
    })
    challenge_train_sample: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 500
    })
    challenge_validation_sample: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 500
    })
    challenge_test_scramble: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 500
    })
})

## Retrieve one entry from a "train" subset

In [7]:
train_instance = data['train'][0]
train_instance

{'concept_set_id': 0,
 'concepts': ['mountain', 'ski', 'skier'],
 'gem_id': 'common_gen-train-0',
 'gem_parent_id': 'common_gen-train-0',
 'references': [],
 'target': 'Skier skis down the mountain'}

## Retrieve a target sentence from this entry object

In [8]:
concept = train_instance.get("target")
concept

'Skier skis down the mountain'

# Test the filter

## Load a filter class from a corresponding ditectory and try the example from its ReadMe

In [9]:
from filters.gender_bias import GenderBiasFilter

f = GenderBiasFilter("en")
sentences = [ "He is going to make a cake.",
              "She is going to program",
              "Nobody likes washing dishes",
              "He agreed to help him" ]

f.filter(sentences)

True

## Take a subset of target sentences from CommonGem dataset 


In [11]:
def get_sentences(split_name, num_iter):
    sentences = []
    for i in range(num_iter):
        sentence = data[split_name][i].get("target")
        sentences.append(sentence)
    return sentences

sentences = get_sentences('train', 10)
sentences

['Skier skis down the mountain',
 'A skier is skiing down a mountain.',
 'Three skiers are skiing on a snowy mountain.',
 'The dog is wagging his tail.',
 'A dog wags his tail at the boy.',
 'a dog wags its tail with its heart',
 'woman paddling canoe on a lake',
 'paddle an open canoe along lake .',
 'a man paddles his canoe on the lake.',
 'a train pulls into station']

## Apply a filter to this small subset

In [15]:
f.filter(sentences)

True

## Explore statistics which make a filter returning the presence of gender bias

In [13]:
def get_stats(text, language):
    flagged_sentences = f.flag_sentences(text, language)
    feminine, masculine, neutral = f.count_genders(flagged_sentences)
    print("Feminine flagged sentences:", feminine)
    print("Masculine flagged sentences:", masculine)
    print("Neutral flagged sentences:", neutral)
    
get_stats(sentences, "en")

Feminine flagged sentences: 1
Masculine flagged sentences: 3
Neutral flagged sentences: 6


## Retrieve the sentences, corresponding to the gathered statistics

In [14]:
def get_phrases(text, language):
    flagged_sentences = f.flag_sentences(text, language)
    feminine_group, masculine_group, neutral_group = f.sort_groups(flagged_sentences)    
    return feminine_group, masculine_group, neutral_group

feminine_group, masculine_group, neutral_group = get_phrases(sentences, "en")

print("This is a feminine group:", feminine_group)
print("This is a masculine group:", masculine_group)
print("This is a neutral group:", neutral_group)

This is a feminine group: ['woman paddling canoe on a lake']
This is a masculine group: ['The dog is wagging his tail.', 'A dog wags his tail at the boy.', 'a man paddles his canoe on the lake.']
This is a neutral group: ['Skier skis down the mountain', 'A skier is skiing down a mountain.', 'Three skiers are skiing on a snowy mountain.', 'a dog wags its tail with its heart', 'paddle an open canoe along lake .', 'a train pulls into station']


## Reduce the number of sentences from the majority group, in order to balance it with the minority group

In [16]:
reduced_masculine_group = masculine_group[2:3]
reduced_masculine_group

['a man paddles his canoe on the lake.']

## Create a balanced version of the subset

In [34]:
def create_balanced(split_name, num_iter, feminine_group, masculine_group, neutral_group):
    balanced_sentences =  []
    balanced_entries = []
    counter = 0

    for i in range(num_iter):
        train_instance = data[split_name][i].get("target")
        if train_instance in feminine_group:
            balanced_entry = {
                "gem_id": "common_gen-challenge_train_{}".format(counter),
                "gem_parent_id": data[split_name][i].get("gem_parent_id"),
                "target": [data[split_name][i].get("target")]
            }
            counter = counter + 1
            balanced_entries.append(balanced_entry)
            balanced_sentences.append(train_instance)

        elif train_instance in masculine_group:
            balanced_entry = {
                "gem_id": "common_gen-challenge_train_{}".format(counter),
                "gem_parent_id": data[split_name][i].get("gem_parent_id"),
                "target": [data[split_name][i].get("target")]
            }
            counter = counter + 1
            balanced_entries.append(balanced_entry)
            balanced_sentences.append(train_instance)

        elif train_instance in neutral_group:
            balanced_entry = {
                "gem_id": "common_gen-challenge_train_{}".format(counter),
                "gem_parent_id": data[split_name][i].get("gem_parent_id"),
                "target": [data[split_name][i].get("target")]
            }
            counter = counter + 1
            balanced_entries.append(balanced_entry)
            balanced_sentences.append(train_instance)

    print("The length of the balanced array is", len(balanced_sentences))
    return balanced_entries, balanced_sentences

balanced_entries, balanced_sentences = create_balanced("train", 10, feminine_group, reduced_masculine_group, neutral_group)

The length of the balanced array is 8


## Verify that the filter returns "False" with the balanced selection of sentences

In [37]:
f.filter(balanced_sentences)

False

## Verify the final array of objects

In [35]:
balanced_entries

[{'gem_id': 'common_gen-challenge_train_0',
  'gem_parent_id': 'common_gen-train-0',
  'target': ['Skier skis down the mountain']},
 {'gem_id': 'common_gen-challenge_train_1',
  'gem_parent_id': 'common_gen-train-1',
  'target': ['A skier is skiing down a mountain.']},
 {'gem_id': 'common_gen-challenge_train_2',
  'gem_parent_id': 'common_gen-train-2',
  'target': ['Three skiers are skiing on a snowy mountain.']},
 {'gem_id': 'common_gen-challenge_train_3',
  'gem_parent_id': 'common_gen-train-5',
  'target': ['a dog wags its tail with its heart']},
 {'gem_id': 'common_gen-challenge_train_4',
  'gem_parent_id': 'common_gen-train-6',
  'target': ['woman paddling canoe on a lake']},
 {'gem_id': 'common_gen-challenge_train_5',
  'gem_parent_id': 'common_gen-train-7',
  'target': ['paddle an open canoe along lake .']},
 {'gem_id': 'common_gen-challenge_train_6',
  'gem_parent_id': 'common_gen-train-8',
  'target': ['a man paddles his canoe on the lake.']},
 {'gem_id': 'common_gen-challenge

## Write a balanced version to a json file

In [52]:
import json

with open('common_gen_train_gender_bias_test.json', 'w') as outfile:
    json.dump(balanced_entries, outfile)

# Apply the process to multiple entries

In [23]:
sentences = get_sentences('validation', len(data["validation"]))
f.filter(sentences)

True

In [24]:
get_stats(sentences, "en")

Feminine flagged sentences: 192
Masculine flagged sentences: 344
Neutral flagged sentences: 457


In [77]:
feminine_group, masculine_group, neutral_group = get_phrases(sentences, "en")
reduced_masculine_group = masculine_group[:192]

In [78]:
balanced, balanced_sentences = create_balanced('validation', len(data["validation"]), feminine_group, reduced_masculine_group, neutral_group)

The length of the balanced array is 841


In [ ]:
with open('common_gen_train_gender_bias.json', 'w') as outfile:
    json.dump(balanced_entries, outfile)